# Feature Engineering Recipe

## Initialization

### Authentication

In [ ]:
from tmo import TmoClient

# Docker image constant to avoid duplication
DOCKER_IMAGE = (
    "artifacts.td.teradata.com/tdproduct-docker-snapshot/avmo/vmo-python-base:3.11.4"
)

client = TmoClient()

client.project_id = "23e1df4b-b630-47a1-ab80-7ad5385fcd8d"

### Fetch Default Dataset Connection

In [ ]:
# New API, allows users do fetch a default dataset connection
# (notice it only works for real users, service account probably don't have personal connections)

default_connection = client.user_attributes().get_default_connection()
default_connection
default_connection_id = default_connection["value"]["defaultDatasetConnectionId"]
default_connection_id

## Import Task

In [ ]:
import uuid

artefacts = ["./artifacts/requirements.txt", "./artifacts/task.py"]

artefact_id = uuid.uuid4()
client.trained_model_artefacts().upload_artefacts(artefact_id, artefacts)

import_request = {
    "artefactImportId": str(artefact_id),
    "name": "FE Task",
    "description": "Feature Engneering Task",
    "language": "PYTHON",
    "functionName": "run_task",
    "metadata": {
        "automation": {
            "resources": {
                "cpu": "2",
                "memory": "2g",
            },
            "engine": "PYTHON",
            "dockerImage": DOCKER_IMAGE,
        },
        "resourceTemplate": "Default",
    },
}

fe = client.feature_engineering()

task = fe.import_task(import_request)
task_id = task["metadata"]["featureEngineeringTask"]["id"]
task_id

## Run Task

In [ ]:
# Payload to run the task
run_request = {
    "automation": {
        "dockerImage": DOCKER_IMAGE,
        "resources": {"memory": "2g", "cpu": "2"},
    },
    "datasetConnectionId": default_connection_id,
}

execution = fe.run(task_id, run_request)
execution

## Approve Task

In [ ]:
fe.approve(task_id, "LGTM")

## Deploy Task

In [ ]:
# Payload to deploy the task
deploy_request = {
    "engineType": "DOCKER_BATCH",
    "engineTypeConfig": {
        "dockerImage": DOCKER_IMAGE,
        "engine": "python-batch",
        "resources": {"memory": "500m", "cpu": "0.5"},
    },
    "language": "python",
    "datasetConnectionId": default_connection_id,
    "cron": "@manual",
}

fe.deploy(task_id, deploy_request)

### Fetch Deployment Id

In [ ]:
# fetch the deployment by task ID

response = client.deployments().find_active_by_task_id(task_id, "expandDeployment")
deployment_id = response["_embedded"]["deployments"][0]["id"]

deployment_id

## Retire Task

In [ ]:
# Payload to retire the task
retire_request = {"deploymentId": deployment_id}

fe.retire(task_id, retire_request)